<a href="https://colab.research.google.com/github/capastomm/capastomm/blob/main/cc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import csv
import numpy as np

#funzione che carica un file con 18 importi per riga, dal più vecchio al più recente
def loadcsv(filename):
  saldi_cc=[]
  with open(filename) as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    next(reader)
    for row in reader:
      saldo_cc=[]
      #ripete la decodifica dell'importo sui 18 mesi
      for i in range(18):
        #l'importo viene mappato su una scala logaritma
        if row[i]=='': 
          importo_dec=0
        else:
          importo=float(row[i].replace(',','.'))
          importo_abs=np.abs(float(importo))
          importo_sign=np.sign(float(importo))
          if (importo_abs < 100):
            importo_dec=importo_sign if importo_abs > 0 else 1
          else:           
            importo_log=int(np.log10(importo_abs))-1
            importo_dec=(importo_log*2 + (1 if importo_abs / np.power(10,importo_log) > 5.0 else 0))*importo_sign
        saldo_cc.append(int(importo_dec))
      saldi_cc.append(np.array(saldo_cc))
  saldi_cc = np.array(saldi_cc)
  min_importo_dec=saldi_cc.min()
  max_importo_dec=saldi_cc.max()
  #si rimappano i saldi su un intervallo positivo
  saldi_cc=saldi_cc-min_importo_dec
  #si tiene traccia della lunghezza del dizionario degli importi decodificati
  input_dim=max_importo_dec-min_importo_dec+1
  content_dic={"saldi_cc":saldi_cc,"input_dim":input_dim}
  return content_dic

#carica il file degli indicatori di solvibilità a 12 mesi
def loadcsv_y(filename):
  ind_def=[]
  with open(filename) as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    next(reader)
    for row in reader:
      ind_def.append(int(row[0]))
  ind_def = np.array(ind_def)
  return ind_def

#carica il file con i 18 saldi
csvcontent=loadcsv('./cc.csv')
#carica il file con i 18 importi cumulati delle movimentazioni  
csvcontent_mov_av=loadcsv('./cc mov av.csv')  
#carica il file con i 18 importi dei fidi
csvcontent_acc=loadcsv('./cc acc.csv')  
#carica gli indicatori di solvibilità
ind_def=loadcsv_y('./ind def.csv')

In [35]:
#si definisce una metrica che esegue il calcolo dell'assorbimento patrimoniale data la probabilità di insolvenza stimata
import tensorflow_probability as tfp

dist=tfp.distributions.Normal(0,1)

def compute_rwa(x):
  return dist.cdf(dist.quantile(x)/tf.math.sqrt (0.85)+tf.math.sqrt (0.15/0.85)*dist.quantile(0.999))

class RWA(tf.keras.metrics.Metric):
  def __init__(self, name='RWA', **kwargs):
    super(RWA, self).__init__(name=name, **kwargs)
    self.rwa = self.add_weight(name='tp', initializer='zeros')


  def update_state(self, y_true, y_pred, sample_weight=None):
    values=tf.map_fn(compute_rwa, y_pred)
    self.rwa.assign_add(tf.reduce_sum(values))

  def result(self):
    return self.rwa

In [36]:
#si definisce una rete multistrato fully connected che elabora i 18 valori di accordato,saldo e movimentazione
output_dim=int(csvcontent["input_dim"]/2)
output_dim_mov_av=int(csvcontent_mov_av["input_dim"]/2)
output_dim_acc=int(csvcontent_acc["input_dim"]/2)
#si definiscono i layer della DNN
inp=tf.keras.Input(shape=(18),dtype=tf.int32)
inp_av=tf.keras.Input(shape=(18),dtype=tf.int32)
inp_acc=tf.keras.Input(shape=(18),dtype=tf.int32)
embl=tf.keras.layers.Embedding(csvcontent["input_dim"],output_dim)(inp)
embl_av=tf.keras.layers.Embedding(csvcontent_mov_av["input_dim"],output_dim_mov_av)(inp_av)
embl_acc=tf.keras.layers.Embedding(csvcontent_acc["input_dim"],output_dim_acc)(inp_acc)
conc_l=tf.keras.layers.Concatenate()([embl,embl_av,embl_acc])
flt_l=tf.keras.layers.Flatten()(conc_l)
dns_l=tf.keras.layers.Dense(18*(output_dim+output_dim_mov_av+output_dim_acc)/2,activation="relu")(flt_l)
dns_l2=tf.keras.layers.Dense(18*(output_dim+output_dim_mov_av+output_dim_acc)/4,activation="relu")(dns_l)
dns_fin=tf.keras.layers.Dense(1,activation="sigmoid")(dns_l2)
ml=tf.keras.Model(inputs=[inp,inp_av,inp_acc],outputs=[dns_fin])
#parametri di training
ml.compile(optimizer="adam",loss=tf.keras.losses.binary_crossentropy,metrics=[tf.keras.metrics.AUC(),RWA()])
ml.summary()
#esecuzione del training
history=ml.fit(x=[csvcontent["saldi_cc"],csvcontent_mov_av["saldi_cc"],csvcontent_acc["saldi_cc"]],y=ind_def,validation_split=0.3,epochs=5)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 18)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 18)]         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 18)]         0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 18, 11)       253         input_19[0][0]                   
____________________________________________________________________________________________

In [37]:
#approccio classico: si stimano 3 regressioni logistiche rispetto ai valori dell'ultimo mese di saldo,movimentazioni e accordato
#si esegue poi un'integrazione dei 3 modelli con un ulteriore logistica

#regressione logistica rispetto al saldo ultimo mese
ml=tf.keras.models.Sequential([
  tf.keras.layers.CategoryEncoding(num_tokens=csvcontent["input_dim"],output_mode="multi_hot")
  ,tf.keras.layers.Dense(1)
])
ml.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))#,metrics=[tf.keras.metrics.AUC()])
ml.fit(x=csvcontent["saldi_cc"][:,-1],y=ind_def,validation_split=0.3,epochs=1)
ml.trainable=False

#regressione logistica rispetto alla movimentazione dell'ultimo mese
ml_av=tf.keras.models.Sequential([
  tf.keras.layers.CategoryEncoding(num_tokens=csvcontent_mov_av["input_dim"],output_mode="multi_hot")
  ,tf.keras.layers.Dense(1)
])
ml_av.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))#,metrics=[tf.keras.metrics.AUC()])
ml_av.fit(x=csvcontent_mov_av["saldi_cc"][:,-1],y=ind_def,validation_split=0.3,epochs=1)
ml_av.trainable=False

#regressione logistica rispetto all'accordato dell'ultimo mese
ml_acc=tf.keras.models.Sequential([
  tf.keras.layers.CategoryEncoding(num_tokens=csvcontent_acc["input_dim"],output_mode="multi_hot")
  ,tf.keras.layers.Dense(1)
])
ml_acc.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))#,metrics=[tf.keras.metrics.AUC()])
ml_acc.fit(x=csvcontent_acc["saldi_cc"][:,-1],y=ind_def,validation_split=0.3,epochs=1)
ml_acc.trainable=False

#integrazione dei 3 moduli con un ulteriore logistica
conc_l=tf.keras.layers.Concatenate()([ml.output,ml_av.output,ml_acc.output])
dns_fin=tf.keras.layers.Dense(1,activation="sigmoid")(conc_l)
ml_fin=tf.keras.Model(inputs=[ml.input,ml_av.input,ml_acc.input],outputs=[dns_fin])
ml_fin.compile(optimizer="adam",loss=tf.keras.losses.binary_crossentropy,metrics=[tf.keras.metrics.AUC(),RWA()])
history=ml_fin.fit(x=[csvcontent["saldi_cc"][:,-1],csvcontent_mov_av["saldi_cc"][:,-1],csvcontent_acc["saldi_cc"][:,-1]],y=ind_def,validation_split=0.3,epochs=2)

#attenzione alla metrica RWA(assorbimento patrimoniale). L'assorbimemto patrimoniale del validation set è di circa il 20% più elevato

6955/6955 [==============================] - 12s 2ms/step - loss: 0.1666 - val_loss: 0.0764
Epoch 1/2
6955/6955 [==============================] - 35s 5ms/step - loss: 0.1096 - auc_7: 0.6359 - RWA: 37194.0898 - val_loss: 0.0722 - val_auc_7: 0.7006 - val_RWA: 16254.5342
Epoch 2/2
6955/6955 [==============================] - 33s 5ms/step - loss: 0.1027 - auc_7: 0.6968 - RWA: 40688.2930 - val_loss: 0.0723 - val_auc_7: 0.7147 - val_RWA: 17729.2539


In [40]:
print(tf.shape(ind_def))

tf.Tensor([943072], shape=(1,), dtype=int32)
